In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten
from keras.optimizers import Adam ,RMSprop
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization, Dense, Dropout, Flatten, Conv2D, MaxPool2D

In [ ]:
X_train = pd.read_csv("train.csv")
X_test = pd.read_csv("test.csv")

In [ ]:
y_train = X_train["label"]
X_train = X_train.drop(labels = ["label"],axis = 1) 


In [ ]:
label = Counter(y_train)
plt.bar(label.keys(),label.values())

In [ ]:
X_train = X_train.values.astype("float32")
X_test = X_test.values.astype("float32")
y_train = y_train.values.astype("int32")

In [ ]:
X_train.shape , X_test.shape

In [ ]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
u,index = np.unique(y_train,return_index=True)

In [ ]:
a = 0
for i in range(len(index)):
    a = a + 1
    plt.subplot(3, 4, a)
    plt.imshow(X_train[index[i]][:,:,0], cmap=plt.get_cmap('gray'))
    plt.title(y_train[index[i]])

    

In [ ]:
from keras.utils.np_utils import to_categorical
y_train= to_categorical(y_train)
num_classes = y_train.shape[1]

In [ ]:
X = X_train
y = y_train
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

In [ ]:
datagen = ImageDataGenerator(
        zca_whitening=False,  # apply ZCA whiteningTrue
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.08,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.08,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=True)  # randomly flip images


datagen.fit(X_train)
train_iterator = datagen.flow(X_train,y_train, batch_size=64)

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))


model.add(Flatten())
model.add(Dense(1000, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(1000, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', patience = 25)
callback = [checkpoint, es]

In [ ]:
history = model.fit_generator(generator=train_iterator, steps_per_epoch=len(train_iterator), epochs=100, 
                    validation_data = (X_val, y_val), callbacks = callback)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(50)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.load_weights("weights.best.hdf5")

In [ ]:
pred = model.predict_classes(X_test, verbose=1)

In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub['Label'] = pred
sub.to_csv("output.csv", index=False)
sub.head()